### establishing connection

In [1]:
from dotenv import load_dotenv
from snowflake.snowpark.session import Session
import os

load_dotenv()

connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
    "role": os.getenv("SNOWFLAKE_ROLE"),
}

snowpark_session = Session.builder.configs(connection_parameters).create()

### testing the connection

In [2]:
from snowflake.cortex import Complete

print(Complete("mistral-large2", "can you understand images ?"))

 I can't directly view or understand images. However, I can help you with information, answer questions, or provide explanations based on text descriptions of images. If you describe an image to me, I can certainly assist you with that!


### building a simple cortex retriever

In [3]:
import os
from snowflake.core import Root
from typing import List

class CortexSearchRetriever:

    def __init__(self, snowpark_session: Session, limit_to_retrieve: int = 3):
        self._snowpark_session = snowpark_session
        self._limit_to_retrieve = limit_to_retrieve

    def retrieve(self, query: str) -> List[str]:
        root = Root(self._snowpark_session)
        cortex_search_service = (
            root.databases[os.getenv("SNOWFLAKE_DATABASE")]
            .schemas[os.getenv("SNOWFLAKE_SCHEMA")]
            .cortex_search_services["CC_SEARCH_SERVICE_CS"]
        )
        resp = cortex_search_service.search(
            query=query,
            columns=["TEXT_CONTENT"],
            limit=self._limit_to_retrieve,
        )

        if resp.results:
            return [curr["TEXT_CONTENT"] for curr in resp.results]
        else:
            return []

### testing the retriever

In [4]:
retriever = CortexSearchRetriever(snowpark_session=snowpark_session, limit_to_retrieve=4)

retrieved_context = retriever.retrieve(query="how was my christmas on December 25, 2024?")

In [ ]:
retrieved_context

### instead of using actual dates in the query , often users might use reference dates. so creating a function to handle it.

In [6]:
from datetime import datetime

def get_current_date_info():
    current = datetime.now()
    return {
        'date': current.strftime('%Y-%m-%d'),
        'day': current.strftime('%A'),
        'full_date': current.strftime('%A, %B %d, %Y'),
        'time': current.strftime('%I:%M %p')
    }
def standardize_dates(query):
  
    current_date = get_current_date_info()
  
    prompt = f"""Current date is {current_date['full_date']}.
    Convert any relative date references (today, tomorrow, next week, etc.) in this query to actual dates.if there is nothing relative, ignore the date provided, and just provide the query as it is.
    Original query: "{query}"
    Only output the converted query with no explanations or additional text."""
  
    # Make LLM call using existing Snowflake Mixtral integration
    response = Complete("mistral-large2", prompt)
    return response


In [ ]:
standard_query = standardize_dates("how was my christmas this year?")
standard_query

### starting trulens session

In [ ]:
from trulens.core import TruSession
from trulens.connectors.snowflake import SnowflakeConnector

tru_snowflake_connector = SnowflakeConnector(snowpark_session=snowpark_session)

tru_session = TruSession(connector=tru_snowflake_connector)

### building simple rag

In [9]:
from trulens.apps.custom import instrument


class RAG_from_scratch:

    def __init__(self):
        self.retriever = CortexSearchRetriever(snowpark_session=snowpark_session, limit_to_retrieve=4)

    @instrument
    def retrieve_context(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        standard_query = standardize_dates(query)
        return self.retriever.retrieve(standard_query)

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        prompt = f"""You are a personal AI assistant who helps the user recall and elaborate on their past thoughts, plans, and discussions.
        You have access to the user's personal notes and memories.

        Context of previous discussions:
        <context>
        {context_str}
        </context>

        User's current question: {query}

        Based on the context and your understanding, provide a helpful and precise response.
        If the context directly addresses the question, use those details.
        If not, respond based on the most relevant information available.
        Always be supportive and sound like a trusted personal assistant.

        Respond with a clear, natural text response. Do not use any special formatting or JSON structure.
        """
        return Complete("mistral-large2", prompt)

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve_context(query)
        return self.generate_completion(query, context_str)


rag = RAG_from_scratch()

decorating <function RAG_from_scratch.retrieve_context at 0x000001E99DCED090>
decorating <function RAG_from_scratch.generate_completion at 0x000001E99DCED000>
decorating <function RAG_from_scratch.query at 0x000001E99DCECD30>
adding method <class '__main__.RAG_from_scratch'> retrieve_context __main__
adding method <class '__main__.RAG_from_scratch'> generate_completion __main__
adding method <class '__main__.RAG_from_scratch'> query __main__


#### to evaluate our app, let's use the rag triad which consists of three evaluation functions i.e context relevance ,groundness and answer relevance

In [10]:
from trulens.providers.cortex.provider import Cortex
from trulens.core import Feedback
from trulens.core import Select
import numpy as np

provider = Cortex(
    snowpark_session,
    model_engine="mistral-large2",
)

f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons, name="Groundedness")
    .on(Select.RecordCalls.retrieve_context.rets[:].collect())
    .on_output()
)

f_context_relevance = (
    Feedback(provider.context_relevance, name="Context Relevance")
    .on_input()
    .on(Select.RecordCalls.retrieve_context.rets[:])
    .aggregate(np.mean)
)

f_answer_relevance = (
    Feedback(provider.relevance, name="Answer Relevance")
    .on_input()
    .on_output()
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retrieve_context.rets[:].collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.retrieve_context.rets[:] .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [11]:
from trulens.apps.custom import TruCustomApp

tru_rag = TruCustomApp(
    rag,
    app_name="memex",
    app_version="simple",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
    )

instrumenting <class '__main__.RAG_from_scratch'> for base <class '__main__.RAG_from_scratch'>
	instrumenting retrieve_context
	instrumenting generate_completion
	instrumenting query
skipping base <class 'object'> because of class
skipping base <class '__main__.CortexSearchRetriever'> because of class
skipping base <class 'object'> because of class


### testing  our rag system

In [12]:
prompts = ["how was my christmas this year ?", "how was my last saturday"]

In [ ]:
with tru_rag as recording:
  for prompt in prompts:
        rag.query(prompt)


In [16]:
tru_session.get_leaderboard()

,,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_name,app_version,,,,,
memex,simple,1.0,0.166667,0.8,8.154223,0.113728


#### The answer relevancy and groundness scores show that the response from our RAG system is highly relevant to the question and grounded (i.e., no hallucinations). However, the context relevance score is only 0.16, indicating poor retriever performance. It might be pulling in some unrelated context/information along with relevant chunks, but the ratio of unrelated content seems to be much higher than the related one.

### since some non-relevant context are being pulled by the retriever , let's implement a guardrail using Trulens based on context relevance score

In [17]:
from trulens.core.guardrails.base import context_filter

f_context_relevance_score = Feedback(
    provider.context_relevance, name="Context Relevance"
)


class filtered_RAG_from_scratch(RAG_from_scratch):

    @instrument
    @context_filter(f_context_relevance_score, 0.65, keyword_for_prompt="query")
    def retrieve_context(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        standard_query = standardize_dates(query)
        return self.retriever.retrieve(standard_query)


filtered_rag = filtered_RAG_from_scratch()

decorating <function context_filter.__call__.<locals>.wrapper at 0x000001E9ACF7D990>
adding method <class '__main__.filtered_RAG_from_scratch'> retrieve_context __main__


In [18]:
tru_filtered_rag = TruCustomApp(
    filtered_rag,
    app_name="memex",
    app_version="filtered",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
)

instrumenting <class '__main__.filtered_RAG_from_scratch'> for base <class '__main__.filtered_RAG_from_scratch'>
	instrumenting retrieve_context
	instrumenting generate_completion
	instrumenting query
instrumenting <class '__main__.filtered_RAG_from_scratch'> for base <class '__main__.RAG_from_scratch'>
	instrumenting retrieve_context
	instrumenting generate_completion
	instrumenting query
skipping base <class 'object'> because of class
skipping base <class '__main__.CortexSearchRetriever'> because of class
skipping base <class 'object'> because of class


Object (of type list is a sequence containing more than one dictionary. Lookup by item or attribute `rets` is ambiguous. Use a lookup by index(es) or slice first to disambiguate.
Object (of type list is a sequence containing more than one dictionary. Lookup by item or attribute `rets` is ambiguous. Use a lookup by index(es) or slice first to disambiguate.
Object (of type list is a sequence containing more than one dictionary. Lookup by item or attribute `rets` is ambiguous. Use a lookup by index(es) or slice first to disambiguate.
Object (of type list is a sequence containing more than one dictionary. Lookup by item or attribute `rets` is ambiguous. Use a lookup by index(es) or slice first to disambiguate.
Object (of type list is a sequence containing more than one dictionary. Lookup by item or attribute `rets` is ambiguous. Use a lookup by index(es) or slice first to disambiguate.
Object (of type list is a sequence containing more than one dictionary. Lookup by item or attribute `rets

In [19]:
prompts = ["how was my christmas this year ?", "how was my last saturday"]

In [ ]:
with tru_filtered_rag as recording:
    for prompt in prompts:
        filtered_rag.query(prompt)

In [24]:
tru_session.get_leaderboard()

c:\Users\User\OneDrive\Desktop\snowflake\myenv\lib\site-packages\trulens\feedback\llm_provider.py:1521: UserWarning: Failed to process and remove trivial statements. Proceeding with all statements.
  warnings.warn(


Answer Relevance  Context Relevance  Groundedness  \
app_name app_version                                                      
memex    filtered                  1.0           0.888889      1.000000   
         simple                    1.0           0.250000      0.866667   

                        latency  total_cost  
app_name app_version                         
memex    filtered     10.286708    0.073477  
         simple        8.154223    0.113728

### Now, the app is much better. The context relevance score has now imporved significantly from (0.25 to 0.88) with the use of Trulens guardrail. This means, only the relevant context are being pulled and used for generating the answer, improving the quality of our rag system (memex)..